In [1]:
# 前準備
!python --version

Python 3.6.1


In [3]:
import os
print( os.listdir(os.path.normpath("./")) )
print( os.listdir(os.path.normpath("./dataset/")) )

['.gitignore', '.ipynb_checkpoints', 'chapter02.ipynb', 'chapter03.ipynb', 'chapter04.ipynb', 'chapter05.ipynb', 'chapter06.ipynb', 'chapter07.ipynb', 'chapter08.ipynb', 'chapter09.ipynb', 'dataset', 'env', 'README.md']
['data_documentation.txt', 'document_word_data.json', 'document_word_data_pnoun.json', 'kc_house_data.csv', 'mix20_rand700_tokens.zip', 'README', 'tokens', 'user_topic_follow_dummy.csv', 'Wholesale_customers_data.csv']


In [4]:
# dataの読み込みとモジュールのインポート
def set_locale():
    default = os.environ.get('LC_ALL')
    print( "Your default locale is", default )
    if default is None:
        os.environ.setdefault('LC_ALL', 'ja_JP.UTF-8')
        print( "Your locale is set as ja_JP.UTF-8" )

set_locale()

Your default locale is None
Your locale is set as ja_JP.UTF-8


In [5]:
import glob

neg_files = glob.glob( os.path.normpath("./dataset/tokens/neg/*") )
pos_files = glob.glob( os.path.normpath("./dataset/tokens/pos/*") )
print(neg_files[0:2])
print(pos_files[0:2])

['dataset/tokens/neg/cv000_tok-9611.txt', 'dataset/tokens/neg/cv001_tok-19324.txt']
['dataset/tokens/pos/cv000_tok-11609.txt', 'dataset/tokens/pos/cv001_tok-10180.txt']


In [6]:
import sys

def text_reader(file_path):
    python_version = sys.version_info.major
    
    if python_version >= 3:
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                print(line)
    else:
        with open(file_path, 'r') as f:
            for line in f:
                print(line)

In [7]:
text_reader(neg_files[11])

* ( out of 4 = poor ) 1995 , g , 90 minutes [1 hour , 30 minutes] [comedy] starring : michael roescher ( hank royce ) , kristy young ( jinnie sue macallister ) , justin garms ( voice of gordy ) , james donadio ( gilbert sipes ) , written by leslie stevens , jay sommers , dick chevillat , produced by sybil robson , directed by mark lewis . " gordy " is not a movie , it is a 90-minute-long " sesame street " skit , and a very bad one at that . this movie is so stupid and dumb that it's depressing to think that some hollywood executives actually gave this the green light , and even more surprising is the fact that this is a disney movie . i'm sure children are the target audience of this movie , but only kids under the age of five may be able to tolerate it . it is the story of a farm a piglet named gordy ( voiced by garms ) , whose family has been taken away to " up north , " which we know means death . of course we can hear the animals talk to each other , and they actually went to the t

今回使うモジュールの情報をまとめておきます。
詳細な中身などに関してはご自身で調べてみてください。
- matplotlib : グラフなどを描写する
http://matplotlib.org/
- pandas : dataframeでデータを扱い、集計や統計量算出などがすぐ実行できる
http://pandas.pydata.org/
- collections : pythonで扱えるデータの型を提供する
http://docs.python.jp/3/library/collections.html
- numpy : 行列などの数学的オブジェクトを扱う
http://www.numpy.org/
- sklearn.feature_extraction.DictVectorizer : 辞書データをベクトルの形に変換する
http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.DictVectorizer.html
- sklearnのモデル : SVM, NB, RF
http://scikit-learn.org/stable/tutorial/basic/tutorial.html
- sklearn.grid_search : パラメタの最適な組み合わせ見つける
http://scikit-learn.org/stable/modules/grid_search.html

In [10]:
# import matplotlib # not used in this notebook
# import pandas as pd # not used in this notebook

import collections
import numpy as np

from sklearn.feature_extraction import DictVectorizer

from sklearn import svm, naive_bayes
from sklearn.ensemble import RandomForestClassifier

from sklearn import grid_search

In [11]:
# 特徴ベクトルの作成
def word_counter(string):
    words = string.strip().split()
    count_dict = collections.Counter(words)
    return dict(count_dict)

def get_unigram(file_path):
    result = []
    python_version = sys.version_info.major
    
    if python_version >= 3:
        for file in file_path:
            with open(file, 'r', encoding='utf-8') as f:
                for line in f:
                    count_dict = word_counter(line)
                    result.append(count_dict)
    else:
        for file in file_path:
            with open(file, 'r') as f:
                for line in f:
                    count_dict = word_counter(line)
                    result.append(count_dict)
    
    return result

In [12]:
word_counter("I am YK. I love data analysis using python.")

{'I': 2,
 'YK.': 1,
 'am': 1,
 'analysis': 1,
 'data': 1,
 'love': 1,
 'python.': 1,
 'using': 1}

In [13]:
%%time

DATA_NUM = 700

unigrams_data = get_unigram(neg_files[:DATA_NUM]) + get_unigram(pos_files[:DATA_NUM])


CPU times: user 299 ms, sys: 88.6 ms, total: 388 ms
Wall time: 508 ms


In [14]:
print( unigrams_data[0] )
print( "data size :", sys.getsizeof(unigrams_data) / 1000000, "[MB]" )


{'tristar': 1, '/': 3, '1': 1, ':': 9, '30': 1, '1997': 2, 'r': 1, '(': 6, 'language': 1, ',': 25, 'violence': 1, 'dennis': 3, 'rodman': 7, ')': 6, 'cast': 1, 'jean-claude': 2, 'van': 4, 'damme': 4, ';': 6, 'mickey': 2, 'rourke': 2, 'natacha': 2, 'lindinger': 2, 'paul': 2, 'freeman': 1, 'director': 1, 'tsui': 2, 'hark': 1, 'screenplay': 2, 'dan': 1, 'jakoby': 1, 'mones': 1, 'ripe': 1, 'with': 5, 'explosions': 1, 'mass': 1, 'death': 1, 'and': 17, 'really': 1, 'weird': 1, 'hairdos': 1, "hark's": 1, '"': 22, 'double': 5, 'team': 5, 'must': 1, 'be': 2, 'the': 20, 'result': 2, 'of': 3, 'a': 13, 'tipsy': 1, 'hollywood': 1, 'power': 1, 'lunch': 1, 'that': 10, 'decided': 1, 'needs': 2, 'another': 2, 'notch': 1, 'on': 5, 'his': 4, 'bad': 2, 'movie-bedpost': 1, 'nba': 1, 'superstar': 1, 'should': 2, 'have': 2, 'an': 5, 'acting': 1, 'career': 1, '.': 31, 'actually': 1, 'in': 8, "neither's": 1, 'performance': 1, 'is': 9, 'all': 4, "i've": 1, 'always': 1, 'been': 1, 'one': 2, 'critic': 1, 'to': 20,

In [15]:
%%time
vec = DictVectorizer()
feature_vectors_csr = vec.fit_transform( unigrams_data )

CPU times: user 569 ms, sys: 22.6 ms, total: 592 ms
Wall time: 599 ms


In [16]:
feature_vectors_csr

<1400x44219 sparse matrix of type '<class 'numpy.float64'>'
	with 496525 stored elements in Compressed Sparse Row format>

In [17]:
feature_vectors = vec.fit_transform( unigrams_data ).toarray()
print( "data dimension :", feature_vectors.shape )
print( feature_vectors[0] )
print( "data size :", sys.getsizeof(feature_vectors) / 1000000, "[MB]" )

data dimension : (1400, 44219)
[  0.   0.  22. ...,   0.   0.   0.]
data size : 495.252912 [MB]


In [18]:
# ラベルデータの作成
labels = np.r_[np.tile(0, DATA_NUM), np.tile(1, DATA_NUM)]
print( labels[0], labels[DATA_NUM-1], labels[DATA_NUM], labels[2*DATA_NUM-1]  )

0 0 1 1


In [20]:
# 学習用データとテスト用データの作成方法
np.random.seed(7789)
shuffle_order = np.random.choice( 2*DATA_NUM, 2*DATA_NUM, replace=False )

In [21]:
print( "length :", len(shuffle_order) )
print( "first 10 elements :", shuffle_order[0:10] )

length : 1400
first 10 elements : [1235 1232  910  162  343 1160  221  545 1112 1322]


In [22]:
one_third_size = int( 2*DATA_NUM / 3. )
print( "one third of the length :", one_third_size )

print( "# of '1' in 1st set :", np.sum( labels[ shuffle_order[:one_third_size] ]  ) )
print( "# of '1' in 2nd set :", np.sum( labels[ shuffle_order[one_third_size:2*one_third_size] ]  ) )
print( "# of '1' in 3rd set :", np.sum( labels[ shuffle_order[2*one_third_size:] ]  ) )

one third of the length : 466
# of '1' in 1st set : 227
# of '1' in 2nd set : 233
# of '1' in 3rd set : 240


In [23]:
# モデルを学習して精度を検証
def N_splitter(seq, N):
    avg = len(seq) / float(N)
    out = []
    last = 0.0
    
    while last < len(seq):
        out.append( seq[int(last):int(last + avg)] )
        last += avg
        
    return np.array(out)

In [24]:
N_splitter(range(14), 3)

array([range(0, 4), range(4, 9), range(9, 14)], dtype=object)

### モデルの学習や予測のための関数を定義します。
- train_model : 説明変数とラベルと手法を与えることでモデルを学習する
- predict : モデルと説明変数を与えることでラベルを予測する
- evaluate_model : 予測したラベルと実際の答えの合致数を調べる
- cross_validate : cross_validationを実行する

In [25]:
def train_model(features, labels, method='SVM', parameters=None):
    ### set the model
    if method == 'SVM':
        model = svm.SVC()
    elif method == 'NB':
        model = naive_bayes.GaussianNB()
    elif method == 'RF':
        model = RandomForestClassifier()
    else:
        print("Set method as SVM (for Support vector machine), NB (for Naive Bayes) or RF (Random Forest)")
    ### set parameters if exists
    if parameters:
        model.set_params(**parameters)
    ### train the model
    model.fit( features, labels )
    ### return the trained model
    return model

def predict(model, features):
    predictions = model.predict( features )
    return predictions

def evaluate_model(predictions, labels):
    data_num = len(labels)
    correct_num = np.sum( predictions == labels )
    return data_num, correct_num

def cross_validate(n_folds, feature_vectors, labels, shuffle_order, method='SVM', parameters=None):
    result_test_num = []
    result_correct_num = []
    
    n_splits = N_splitter( range(2*DATA_NUM), n_folds )

    for i in range(n_folds):
        print( "Executing {0}th set...".format(i+1) )
        
        test_elems = shuffle_order[ n_splits[i] ]
        train_elems = np.array([])
        train_set = n_splits[ np.arange(n_folds) !=i ]
        for j in train_set:
            train_elems = np.r_[ train_elems, shuffle_order[j] ]
        train_elems = train_elems.astype(np.integer)

        # train
        model = train_model( feature_vectors[train_elems], labels[train_elems], method, parameters )
        # predict
        predictions = predict( model, feature_vectors[test_elems] )
        # evaluate
        test_num, correct_num = evaluate_model( predictions, labels[test_elems] )
        result_test_num.append( test_num )
        result_correct_num.append( correct_num )
    
    return result_test_num, result_correct_num

In [26]:
N_FOLDS = 3

In [27]:
%%time
ans,corr = cross_validate(N_FOLDS, feature_vectors_csr, labels, shuffle_order, method='SVM', parameters=None)


Executing 1th set...
Executing 2th set...
Executing 3th set...
CPU times: user 10.4 s, sys: 42 ms, total: 10.4 s
Wall time: 10.4 s


In [28]:
print( "average precision : ", np.around( 100.*sum(corr)/sum(ans), decimals=1 ), "%" )

average precision :  63.2 %


In [29]:
%%time

svm_model = train_model(
    features=feature_vectors_csr[shuffle_order[0:950],:]
    , labels=labels[shuffle_order[0:950]]
    , method='SVM'
    , parameters=None
)

CPU times: user 2.35 s, sys: 6.53 ms, total: 2.35 s
Wall time: 2.36 s


In [30]:
print("data : " ,shuffle_order[970:980], "correct label : ", labels[shuffle_order[970:980]])
print( "predict label : ", predict(svm_model, feature_vectors_csr[shuffle_order[970:980], :]) )

data :  [ 490 1276  794  892  504  463   48  289 1218 1137] correct label :  [0 1 1 1 0 0 0 0 1 1]
predict label :  [1 1 0 0 0 1 0 1 1 0]


In [31]:
text_reader(neg_files[490-1])

* directed and co-produced by jodie foster . written by w . d . richter . cast : holly hunter , robert downey , jr . , anne bancroft , dylan mcdermott , charles durning , geraldine chaplin , steve guttenberg , claire danes , cynthia stevenson . distributed by paramount and polygram . the title " home for the holidays " suggests either a feel-good story , a family drama or a comedy , but guesses can be wrong . there was a made-for-tv movie with that title in 1972 which turned out to be a christmastime thriller . now , the current film's distributors have announced it as a comedy , but this description will do only if you stretch it . claudia ( holly hunter ) just lost her job as an art restorer at a museum . ( you wait a long time to find out that the museum is in chicago . ) with some trepidation , claudia leaves behind her sixteen-year old daughter ( we later learn in passing that hunter is unmarried ) and flies to another state to spend thanksgiving with her parents . ( you wait a lo

# パラメタチューニング

In [33]:
%%time

search_parameters = [
    {'kernel': ['rbf'], 'gamma': [1e-2, 1e-3, 1e-4], 'C': [0.1, 1, 10, 100, 1000]},
    {'kernel': ['linear'], 'C': [0.1, 1, 10, 100, 1000]}
]

model = svm.SVC()
clf = grid_search.GridSearchCV(model, search_parameters)
clf.fit( feature_vectors_csr, labels )

CPU times: user 3min 34s, sys: 639 ms, total: 3min 35s
Wall time: 3min 36s


In [34]:
print("best paremters : ", clf.best_params_)
print("best scores : ", clf.best_score_)


best paremters :  {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
best scores :  0.7978571428571428


In [38]:
%%time
ans,corr = cross_validate(N_FOLDS, feature_vectors_csr, labels, shuffle_order, method='SVM', parameters=clf.best_params_)

Executing 1th set...
Executing 2th set...
Executing 3th set...
CPU times: user 10.3 s, sys: 46.9 ms, total: 10.3 s
Wall time: 10.4 s


In [39]:
print( "average precision : ", np.around( 100.*sum(corr)/sum(ans), decimals=1 ), "%" )

average precision :  79.5 %
